<a href="https://colab.research.google.com/github/jjsalaza/Ingenieria-Logistica-ULL/blob/master/logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ingeniería Logística, Grado de Informática, Universidad de La Laguna
---



# Introducción

Principalmente aprendemos a representar problemas reales mediante modelos matemáticos de programación lineal (continua o entera), es decir, del tipo $\max\{ c^Tx: Ax\leq b, x\in Z^n\}$. Luego estos modelos matemáticos serán resueltos mediante herramientas informáticas especializadas, como [Cplex](https://www.ibm.com/analytics/cplex-optimizer), [Xpress](http://www.fico.com/es/products/fico-xpress-optimization), Gurobi, SCIP, GLKP, CBC, GLOP, etc. Aunque dedicamos algo a comprender las metodologías implementadas en estas herramientas, principalmente la asignatura insistirá en la modelización de problemas.


Un ejemplo de modelo matemático es:
\begin{eqnarray}
\max\;\; x+10y& & \\
x+7y&\leq& 17.5  \\
x &\leq& 3.5  \\
x,y&\in& Z
\end{eqnarray}

A fín de no complicarnos con la instalación de licencias comerciales, usamos una librería de optimización de software libre que permite ser llamada desde Python y que ha sideo desarrollada en el proyecto [COIN-OR](https://www.coin-or.org/). Se llama CBC. Además en este mismo proyecto existe un lenguaje que amplía el Python y se llama [PuLP](https://www.coin-or.org/PuLP/). Usando este lenguaje el modelo anterior se resuelve mediante:


In [0]:
!pip install pulp
from pulp import *

prob = LpProblem("MiPrimerProblemaEntero", LpMaximize)
X = LpVariable("X", 0, None,cat='Integer')
Y = LpVariable("Y", 0, None,cat='Integer')
prob += X + 10 * Y
prob += X + 7 * Y <= 17.5
prob += X <= 3.5

prob.solve()

print("Status = ", LpStatus[prob.status])
print("Optimal objective value = ", value(prob.objective))
print("  X = ", X.varValue)
print("  Y = ", Y.varValue)
# prob.writeLP("modelo.lp")
# print(prob)

En este lenguaje PuLP podemos usar varias librerias de optimización. Por defecto usa la de COIN-or. Pero podemos elegir otras usando por ejemplo:
\begin{verbatim}
if liberia == "GUROBI":
			prob.solve(GUROBI(TimeLimit=60,msg=1))
		elif libreria == "CBC":
			prob.solve(PULP_CBC_CMD(msg=1,maxSeconds=60,options=["rens","on","local","on"]))
		else:
			print( libreria + " solver doesn't exists")
			quit()
\end{verbatim}
En este sentido parece un legunaje ideal, pero hay que tener en cuenta que algunas librerías ofrecen funciones no disponibles en otras librerias (por ejemplo, callback functions) con lo que igual tampoco PuLP es tan ideal.

Otra alternativa es usar la nueva herramienta gratuita (¡por ahora!) de Google llamada [OR-tools](https://developers.google.com/optimization/). Ni PuLP ni OR-tools son librerías de optimización, sino extensiones del lenguaje Python para escribir cómodamente modelos matemáticos de Programación Lineal (Entera o Continua) que luego pueden ser resueltas con cualquiera de las librerias citadas (CBC, Cplex, Gurobi,...). Actuan como puente entre quien desea resolver un modelo y quien lo resuelve. Muy cómodo. Muy recomendable. La opción de Google es:

In [0]:
!pip install ortools
from ortools.linear_solver import pywraplp

Tenemos dos modos de escribir los detalles del modelo. O bien coeficiente a coeficiente, o bien fila a fila. Mostramos a continuación ambos:

In [0]:
solver = pywraplp.Solver('MiPrimerProblemaEntero',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# x and y are integer non-negative variables.
x = solver.IntVar(0.0, solver.infinity(), 'x')
y = solver.IntVar(0.0, solver.infinity(), 'y')

solver.Maximize(x + 10 * y)
solver.Add( x + 7 * y <= 17.5 )
solver.Add( x <= 7.5 )  

"""Solve the problem and print the solution."""
result_status = solver.Solve()
# The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL
  
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())
print('Optimal objective value = %d' % solver.Objective().Value())
  
# The value of each variable in the solution.
variable_list = [x, y]

for variable in variable_list:
    print('   %s = %d' % (variable.name(), variable.solution_value()))

In [0]:
solver = pywraplp.Solver('MiPrimerProblemaEntero',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# x and y are integer non-negative variables.
x = solver.IntVar(0.0, solver.infinity(), 'x')
y = solver.IntVar(0.0, solver.infinity(), 'y')

# x + 7 * y <= 17.5
constraint1 = solver.Constraint(-solver.infinity(), 17.5)
constraint1.SetCoefficient(x, 1)
constraint1.SetCoefficient(y, 7)

# x <= 3.5
constraint2 = solver.Constraint(-solver.infinity(), 3.5)
constraint2.SetCoefficient(x, 1)
constraint2.SetCoefficient(y, 0)

# Maximize x + 10 * y.
objective = solver.Objective()
objective.SetCoefficient(x, 1)
objective.SetCoefficient(y, 10)
objective.SetMaximization()

"""Solve the problem and print the solution."""
result_status = solver.Solve()
# The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL
  
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())
print('Optimal objective value = %d' % solver.Objective().Value())
  
# The value of each variable in the solution.
variable_list = [x, y]

for variable in variable_list:
    print('   %s = %d' % (variable.name(), variable.solution_value()))

El anterior ejemplo fue un modelo muy pequeño, con tan sólo 2 variables y 2 restricciones. Son ejemplo numéricos diminutos de nulo interés práctico. En la práctica se plantean problemas más complejos que necesitan ser representado mediante modelos con más variables y restricciones, y en consecuencia donde notación abreviada ayuda a entenderlos mejor y reducir la posibilidad de cometer errores al introducirlos en un ordenador. Veamos un ejemplo de problema mayor en la siguiente sección.

Pero incluso los modelos de programación entera pequeños pueden ser muy difíciles. Aqui lo mostramos con uno con 3 variables:

In [0]:
!pip install ortools
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('ProblemaEnteroDificil',  pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING )

x = solver.IntVar(0.0, solver.infinity(), 'x')
y = solver.IntVar(0.0, solver.infinity(), 'y')
z = solver.IntVar(0.0, solver.infinity(), 'z')

solver.SetTimeLimit( 60 * 1000 )   # le ponemos un tiempo límite de 60 segundos para no calentar el ordenador

solver.Minimize(x)
solver.Add( 75001 * y + 75002 * z == 75000 + 75000 * x )
result_status = solver.Solve()

# The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL
  
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())
print('Optimal objective value = %d' % solver.Objective().Value())
  
# The value of each variable in the solution.
variable_list = [x, y, z]

for variable in variable_list:
    print('   %s = %d' % (variable.name(), variable.solution_value()))
    
print("Time = ", solver.WallTime(), " milliseconds")
print("Iterations = ", solver.Iterations())
print("Nodes = ", solver.nodes())

Mismo problema con Gurobi:

In [0]:
from gurobi import *

modelo = Model("ProblemaEnteroDificil")

x = modelo.addVar(vtype=GRB.INTEGER, name="x")
y = modelo.addVar(vtype=GRB.INTEGER, name="y")
z = modelo.addVar(vtype=GRB.INTEGER, name="z")

modelo.setObjective(x, GRB.MINIMIZE)
modelo.addConstr( 75001 * y + 75002 * z == 75000 + 75000 * x , "c0")
modelo.optimize()

print('Optimal  objective value = %g' % modelo.objVal)
for variable in modelo.getVars():
    print('  %s = %g' % (variable.varName, variable.x))
print('runtime is ',modelo.Runtime)

# Problemas de Asignación

Imaginemos tener 6 trabajadores y 4 tareas, y conocer el costo de $c_{ij}$ de que el trabajador $i$ esté asignado a la tarea $j$. Se busca asignar un trabajador a cada tarea, y que ningún trabajador esté asignado a más de una máquina, de manera que el coste total sea lo menor posible.


In [0]:
cost  =  [[90, 76, 75, 70],
          [35, 85, 55, 65],
          [125, 95, 90, 105],
          [45, 110, 95, 115],
          [60, 105, 80, 75],
          [45, 65, 110, 95]]
num_workers = len(cost)
num_tasks   = len(cost[1])

In [0]:
!pip install ortools
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('ProblemaAsignacion2dimensional',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

x = {}

for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.BoolVar('x[%i,%i]' % (i, j))

# Objective
solver.Minimize(solver.Sum([cost[i][j] * x[i,j] for i in range(num_workers)
                                                  for j in range(num_tasks)]))

# Constraints

# Each worker is assigned to at most 1 task.

for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.

for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)

# resolvemos:  
sol = solver.Solve()

print('Costo total = ', solver.Objective().Value())
print()
for i in range(num_workers):
    for j in range(num_tasks):
        if x[i, j].solution_value() > 0:
            print('Trabajador %d asignado a la tarea %d.  Costo = %d' % (i,j,cost[i][j]))
print()
print("Time = ", solver.WallTime(), " milliseconds")

Si los trabajadores estuviesen en dos equipos, por ejemplo, pares e impares, la solución optima obtenida usa más trabajadores de un equipo que del otro. Podría tener sentido que el problema real también desee una solución donde se eligan exactamente 2 trabajadores de cada equipo. Para ello hay que introducir nuevas restricciones en el modelo matemático:

In [0]:
team1 = [0, 2, 4]
team2 = [1, 3, 5]

# Each team takes on two tasks.

solver.Add(solver.Sum([x[i, j] for i in team1 for j in range(num_tasks)]) <= 2)
solver.Add(solver.Sum([x[i, j] for i in team2 for j in range(num_tasks)]) <= 2)

# resolvemos:  
sol = solver.Solve()

print('Costo total = ', solver.Objective().Value())
print()
for i in range(num_workers):
    for j in range(num_tasks):
        if x[i, j].solution_value() > 0:
            print('Trabajador %d asignado a la tarea %d.  Costo = %d' % (i,j,cost[i][j]))

Como era de esperar, la nueva solución óptima cuesta más que la anterior porque debe cumplir además las nuevas desigualdades.

Terminemos esta sección con un problema de asignación más complejo, conocido como **problema de asignación 3-dimensional**. Pensemos ahora que tenemos 5 profesores, 5 asignaturas y 5 aulas, y queremos asignar un profesor a cada asignatura en un aula, de modo que cada profesor da 1 asignatura en 1 aula, cada asignatura la da 1 profesor en 1 aula, y cada aula imparte 1 asignatura por 1 profesor. Todo queremos que se realice a costo total mínimo. Concretamente se llama problema de asignación 3-dimensional **axial** porque como veremos su modelo matemático se basa en restricciones que tienen 1 índice libre, en contraposición a otro problema que se llama de asignación 3-dimensional **planar** donde su modelo se basa en restricciones que tienen 2 indices libres. Vemos primero el modelo para el problema axial planteado, es decir, asignar profesores, asignaturas y aulas:

In [0]:
import random  
dimension = 100
opciones  = range(dimension)
cost      = [[[random.randint(1,101) for i in opciones] for j in opciones] for k in opciones]

!pip install ortools
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('3APaxial', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
x = {}
for i in opciones:
    for j in opciones:
        for k in opciones:
            x[i,j,k] = solver.BoolVar('x[%i,%i,%i]' % (i,j,k))

# Objective
solver.Minimize( solver.Sum(cost[i][j][k] * x[i,j,k] for i in opciones for j in opciones for k in opciones))

# Constraints

[ solver.Add(solver.Sum(x[i,j,k] for j in opciones for k in opciones) == 1) for i in opciones ]
[ solver.Add(solver.Sum(x[i,j,k] for i in opciones for k in opciones) == 1) for j in opciones ]
[ solver.Add(solver.Sum(x[i,j,k] for i in opciones for j in opciones) == 1) for k in opciones ]
    
# resolvemos:  
sol = solver.Solve()

print('Costo total = ', solver.Objective().Value())
print()
for i in opciones:
    for j in opciones:
        for k in opciones:
            if x[i,j,k].solution_value() > 0:
                print('Prof %d con asignatura %d en aula %d.  Costo = %d' % (i,j,k,cost[i][j][k]))
print()
print("Time = ", solver.WallTime(), " milliseconds")

El modelo del problema de asignación 3-dimensional **planar** es el siguiente. Nótese que **no** es un modelo alternativo para el problema de asignación de profesores, asignatura y aulas. Es el modelo de otro problema diferente. Por eso la solución óptima del modelo anterior y del modelo siguiente no coinciden, ni tienen relación alguna. Si tienes curiosidad por conocer el problema asociado a este otro modelo, te sugiero que estudies el concepto de [Cuadrado Latino](https://es.wikipedia.org/wiki/Cuadrado_latino).

In [0]:
import random  
dimension = 100
opciones  = range(dimension)
cost      = [[[random.randint(1,101) for i in opciones] for j in opciones] for k in opciones]

!pip install ortools
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('3APplanar', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

x = {}
for i in opciones:
    for j in opciones:
        for k in opciones:
            x[i,j,k] = solver.BoolVar('x[%i,%i,%i]' % (i,j,k))

# Objective
solver.Minimize(solver.Sum(cost[i][j][k] * x[i,j,k] for i in opciones for j in opciones for k in opciones))

# Constraints

[ solver.Add(solver.Sum(x[i,j,k] for k in opciones) == 1) for i in opciones for j in opciones ]
[ solver.Add(solver.Sum(x[i,j,k] for j in opciones) == 1) for i in opciones for k in opciones ]
[ solver.Add(solver.Sum(x[i,j,k] for i in opciones) == 1) for j in opciones for k in opciones ]
    
# resolvemos:  
sol = solver.Solve()

print('Costo total = ', solver.Objective().Value())
print()
for i in opciones:
    for j in opciones:
        for k in opciones:
            if x[i,j,k].solution_value() > 0:
                print('Indices: %d  %d  %d.  Costo = %d' % (i,j,k,cost[i][j][k]))
print()
print("Time = ", solver.WallTime(), " milliseconds")

Para resolverlo con Gurobi:

In [1]:
from gurobipy import *
import random

dimension = 100
opciones  = range(dimension)
costo = [[[random.randint(1,101) for i in opciones] for j in opciones] for k in opciones]

modelo=Model("Asignacion3dimensional")
x = {}
for i in opciones:
    for j in opciones:
        for k in opciones:
            x[i,j,k] = modelo.addVar(vtype=GRB.BINARY, name="x[%i,%i,%i]" % (i,j,k))

modelo.modelSense = GRB.MAXIMIZE
modelo.setObjective( quicksum(costo[i][j][k] * x[i,j,k] for i in opciones for j in opciones for k in opciones) )
    
[ modelo.addConstr( quicksum(x[i,j,k] for j in opciones for k in opciones) == 1 ) for i in opciones ]
[ modelo.addConstr( quicksum(x[i,j,k] for i in opciones for k in opciones) == 1 ) for j in opciones ]
[ modelo.addConstr( quicksum(x[i,j,k] for i in opciones for j in opciones) == 1 ) for k in opciones ]
                    
modelo.optimize()
                    
print('Optimal  objective value = %g' % modelo.objVal)
print('runtime is ',modelo.Runtime)

Optimize a model with 300 rows, 1000000 columns and 3000000 nonzeros
Variable types: 0 continuous, 1000000 integer (1000000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 4.54s
Presolved: 300 rows, 1000000 columns, 3000000 nonzeros
Variable types: 0 continuous, 1000000 integer (1000000 binary)
Found heuristic solution: objective 139.0000000
Found heuristic solution: objective 5311.0000000

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0984559e+07   2.999700e+06   0.000000e+00     12s

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity     -5.0984559e+07     12s
       1        300   2.9833590e+12  -4.9984659e+07     12s
       2       1130   2.9698380e+12  -4.8994696e+07     13s
       3       1829   

# Problemas de Transporte

# Problemas de Flujo

# Problemas de Caminos

In [0]:
import sys
import math
import random
#  genera N puntos para luego resolver el problema sobre estos datos
n = 100
random.seed(1)
points = [(random.randint(0,100),random.randint(0,100)) for i in range(n)]

# calculate Euclidean distance and round-towards-zero (truncate)
def distance(points, i, j):
    dx = points[i][0] - points[j][0]
    dy = points[i][1] - points[j][1]
    return math.floor(math.sqrt(dx*dx + dy*dy))

# Dictionary of Euclidean distance between each pair of points
dist = {(i,j) : distance(points,i,j) for i in range(n) for j in range(i)}
  

from gurobipy import *
  
m = Model()
##############################################################################
# inserta tu modelo matemático aqui
##############################################################################
m.optimize()

solution = m.getAttr('x', vars)

selected = [(i,j) for i,j in arcs if solution[i,j] > 0.5]
print filter(lambda x: x[1]>0, solution.iteritems())

print map(lambda (i, j): distance(points, i, j), arcs)

print('')
print('Optimal path: %s' % str(selected))
print('Optimal cost: %g' % m.objVal)
print('')

import matplotlib.pyplot as plt

def plot_path(points, path, style='bo-'):
    "Plot lines to connect a series of points."
    plt.plot(map(lambda x: x[1][0], points.iteritems()), map(lambda x: x[1][1], points.iteritems()), 'bo')
    target = str(len(points))
    plt.plot([points['1'][0],points[target][0]],[points['1'][1],points[target][1]], 'rs')
    plt.plot(map(lambda x: (points[x[0]][0],points[x[1]][0]), path), 
             map(lambda x: (points[x[0]][1],points[x[1]][1]), path), 'b-')
    plt.axis('scaled'); plt.axis('off')
    plt.show()

print points, selected
plot_path(points, selected)

# Problema del Viajante de Comercio

Modelo para el problema del TSP asimétrico con potenciales:

In [4]:
import random
dimension= 20
todos = range(dimension)
otros = range(1,dimension)
DIST= [[random.randint(1,101) for i in todos] for j in todos] 

!pip install ortools
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('ATSP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

x = {}
for i in todos:
    for j in todos:
        if i!=j:
           x[i,j] = solver.BoolVar('x[%i,%i]' % (i,j))
u = {}
for i in otros:
    u[i] = solver.IntVar(0.0, solver.infinity(), 'u[%i]' % i)  

solver.Minimize(solver.Sum( DIST[i][j]*x[i,j] for i in todos for j in todos if i!=j ))
    
[ solver.Add(  solver.Sum(x[i,j] for j in todos if i!=j) == 1  )  for i in todos ]
[ solver.Add(  solver.Sum(x[j,i] for j in todos if i!=j) == 1  )  for i in todos ]
[ solver.Add(  u[j] >= u[i] + x[i,j] - (dimension-2)*(1-x[i,j]) + (dimension-3)*x[j,i]   ) for i in otros for j in otros if j!=i ]
        
solver.Solve()
print('Distancia total de la ruta = ', solver.Objective().Value(),'Kilometros')
print('Ruta de mínima distancia:')
for i in todos:
    for j in todos:
        if i!=j:
            if x[i,j].solution_value() > 0.5 :
                print('De la ciudad %d a la ciudad %d.  Distancia = %d Km'  % (i,j,DIST[i][j]))

Distancia total de la ruta =  199.0 Kilometros
Ruta de mínima distancia:
De la ciudad 0 a la ciudad 3.  Distancia = 11 Km
De la ciudad 1 a la ciudad 10.  Distancia = 18 Km
De la ciudad 2 a la ciudad 4.  Distancia = 3 Km
De la ciudad 3 a la ciudad 6.  Distancia = 12 Km
De la ciudad 4 a la ciudad 7.  Distancia = 17 Km
De la ciudad 5 a la ciudad 2.  Distancia = 11 Km
De la ciudad 6 a la ciudad 18.  Distancia = 8 Km
De la ciudad 7 a la ciudad 0.  Distancia = 12 Km
De la ciudad 8 a la ciudad 16.  Distancia = 3 Km
De la ciudad 9 a la ciudad 14.  Distancia = 1 Km
De la ciudad 10 a la ciudad 8.  Distancia = 9 Km
De la ciudad 11 a la ciudad 19.  Distancia = 5 Km
De la ciudad 12 a la ciudad 13.  Distancia = 15 Km
De la ciudad 13 a la ciudad 11.  Distancia = 1 Km
De la ciudad 14 a la ciudad 5.  Distancia = 27 Km
De la ciudad 15 a la ciudad 12.  Distancia = 2 Km
De la ciudad 16 a la ciudad 15.  Distancia = 19 Km
De la ciudad 17 a la ciudad 9.  Distancia = 15 Km
De la ciudad 18 a la ciudad 1.  Dist

Modelo asimétrico, es decir, con una variable para $(i,j)$ y otra para $(j,i)$

In [ ]:
n = 100
random.seed(1)
points = [(random.randint(0,100),random.randint(0,100)) for i in range(n)]

# calculate Euclidean distance and round-towards-zero (truncate)
def distance(points, i, j):
    dx = points[i][0] - points[j][0]
    dy = points[i][1] - points[j][1]
    return math.floor(math.sqrt(dx*dx + dy*dy))

# Dictionary of Euclidean distance between each pair of points
dist = {(i,j) : distance(points,i,j) for i in range(n) for j in range(n) if i!=j }

In [0]:
import itertools
from gurobipy import *

# Callback: detectando subtours en soluciones enteras

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # creamos el grafo soporte con los arcos que tienen variables positivas
        vals = model.cbGetSolution(model._vars)
        selected = tuplelist((i,j) for i,j in model._vars.keys() if vals[i,j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n:
#            print("nuevo subtour de ",len(tour)," nodos detectado")
            model.cbLazy( quicksum(model._vars[i,j]+model._vars[j,i] for i,j in itertools.combinations(tour, 2)) <= len(tour)-1 )


# Given a tuplelist of edges, find the shortest subtour

def subtour(edges):
    unvisited = list(range(n))
    cycle = range(n+1) # initial length has 1 more city
    while unvisited: # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i,j in edges.select(current,'*') if j in unvisited]
        if len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

# función principal

m = Model()

vars = m.addVars( dist.keys() , obj=dist, vtype=GRB.BINARY, name='a[%d,%d]'%(i,j))

m.addConstrs(vars.sum(i,'*') == 1 for i in range(n))
m.addConstrs(vars.sum('*',i) == 1 for i in range(n))

m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

vals = m.getAttr('x', vars)
selected = tuplelist((i,j) for i,j in vals.keys() if vals[i,j] > 0.5)

tour = subtour(selected)
assert len(tour) == n

print('')
print('Optimal tour: %s' % str(tour))
print('Optimal cost: %g' % m.objVal)
print('')

El mismo modelo pero ahora aprovechando que la matriz de costos es simétrica al ser generadas con distancias Euclídeas, y por tanto con una variable $\{i,j\}$ en lugar de dos variables por pareja de nodos. Esto permite trabajar con la mitad de las variables y termina antes, pero resta generalidad porque con este modelo reducido no podremos resolver ejemplos asimétricos.

In [ ]:
n = 100
random.seed(1)
points = [(random.randint(0,100),random.randint(0,100)) for i in range(n)]

# calculate Euclidean distance and round-towards-zero (truncate)
def distance(points, i, j):
    dx = points[i][0] - points[j][0]
    dy = points[i][1] - points[j][1]
    return math.floor(math.sqrt(dx*dx + dy*dy))

# Dictionary of Euclidean distance between each pair of points
dist = {(i,j) : distance(points,i,j) for i in range(n) for j in range(n) if i!=j }

In [0]:
import sys
import math
import random
import itertools
from gurobipy import *

# Callback: detectando subtours en soluciones enteras

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # creamos el grafo soporte con los arcos que tienen variables positivas
        vals = model.cbGetSolution(model._vars)
        selected = tuplelist((i,j) for i,j in model._vars.keys() if vals[i,j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n:
            print("nuevo subtour de ",len(tour)," nodos detectado")
            model.cbLazy( quicksum(model._vars[i,j] for i,j in itertools.combinations(tour, 2)) <= len(tour)-1 )


# Given a tuplelist of edges, find the shortest subtour

def subtour(edges):
    unvisited = list(range(n))
    cycle = range(n+1) # initial length has 1 more city
    while unvisited: # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i,j in edges.select(current,'*') if j in unvisited]
        if len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

# función principal

m = Model()

vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='e[%d,%d]'%(i,j))
for i,j in vars.keys():
    vars[j,i] = vars[i,j] # edge in opposite direction

m.addConstrs(vars.sum(i,'*') == 2 for i in range(n))

m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

vals = m.getAttr('x', vars)
selected = tuplelist((i,j) for i,j in vals.keys() if vals[i,j] > 0.5)

tour = subtour(selected)
assert len(tour) == n

print('')
print('Optimal tour: %s' % str(tour))
print('Optimal cost: %g' % m.objVal)
print('')

# Problemas de Rutas de Vehículos

# Ejemplos con GUROBI

Para terminar te presento el modelo axial pero en el lenguaje de Gurobi, que es comercial y no podrás ejecutarlo sin licencia:

